In [1]:
import awkward as ak
import numpy as np
import coffea
import os
from plugins import handleData

print(coffea.__version__)
from coffea import util
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import hist

testFiles = [
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_170to300_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/0CDD30E0-B953-E54B-BB7A-BFB45BE6EC93.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/3788BBD3-3B70-BE48-B01A-4FA395E1E0B6.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_470to600_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/F6B6A983-3E2B-7B4D-A2CE-E03AB6E32387.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_600to800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/FDF7FBE4-9226-994A-BD45-29995A3CD62F.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_800to1000_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/2CEB03D7-5B1A-EE46-9C21-13ED3E349AF4.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/DB87A517-0584-1A41-BB18-AAE7C9AAE5B8.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/BFE2E7E9-82CB-FE47-9B43-EBA5C373AF60.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/5213B536-BAA1-CC43-A58A-B1E41E2680F5.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/E8400F91-3216-EA46-A9F1-8CC7EF5261B0.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B21BFDD8-B878-784C-AFB6-5CDE131F5C39.root"
]

# redirector = 'root://xcache/'
# redirector='root://cmsxrootd.fnal.gov/'
# files = handleData('fileset_QCD.json', redirector, testing = True, data = False)
# # files_test = handleData('fileset_QCD.json', redirector, testing = False, data = False)

# events = NanoEventsFactory.from_root(list(files.values())[9][0], schemaclass=NanoAODSchema).events()
# print(files)
#may have to put in weights by hand
# print(events.Generator.weight)

0.7.21


In [2]:
from plugins import *
from dijetProcessor import makeDijetHists
from dask.distributed import Client
import pickle
#### WE'RE MISSING 2016B ver2 -- AK8 PF HLT is missing need to use AK4 trigger isntead

data_bool = True
winterfell = False
dask = True
testing = False
processor = makeDijetHists(data = data_bool)
datastring = "JetHT" if processor.do_gen == False else "QCDsim"
if processor.do_gen==True and winterfell:
    filename = "QCD_flat_files.json"
elif processor.do_gen==True:
    filename = "fileset_QCD.json"
else:
    filename = "datasets_UL_NANOAOD.json"
if testing:
    fname = 'coffeaOutput/dijetHistsTest_wXSscaling_{}_{}_{}_NewHist.pkl'.format(datastring, processor.ptcut, processor.etacut)
else:
    fname = 'coffeaOutput/dijetHists_wXSscaling_{}_{}_{}_NewHist.pkl'.format(datastring, processor.ptcut, processor.etacut)

0.7.21
2.6.3
Data:  True  gen  False


In [ ]:
result = runCoffeaJob(processor, jsonFile = filename, winterfell = winterfell, testing = testing, dask = False, data = not processor.do_gen)
with open(fname, "wb") as f:
    pickle.dump( result, f)

Era:  -UL2016
Era:  HIPM_UL2016
Era:  UL2017
Era:  UL2018
dataset =  /JetHT/Run2016B-ver1_HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD
dataset =  /JetHT/Run2016C-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD
dataset =  /JetHT/Run2016D-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD
dataset =  /JetHT/Run2016E-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD
dataset =  /JetHT/Run2016F-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD
dataset =  /JetHT/Run2016F-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD
dataset =  /JetHT/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD
dataset =  /JetHT/Run2017B-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD
dataset =  /JetHT/Run2017C-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD
dataset =  /JetHT/Run2017D-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD
dataset =  /JetHT/Run2017E-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD
dataset =  /JetHT/Run2017F-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD
dataset =  /JetHT/Run2018A-UL2018_MiniAODv2_NanoAODv9-v2/NANOAOD
dataset =  /JetHT/Run2018B-UL2018_MiniAODv2_NanoAODv9-v1/NANOAOD
Ru

Output()

/opt/conda/lib/python3.8/site-packages/coffea/processor/executor.py:1345: UserWarning: file not found ([ERROR] Server responded with an error: [3008] Unable to read /store/data/Run2016D/JetHT/NANOAOD/HIPM_UL2016_MiniAODv2_NanoAODv9-v2/2530000/B091446A-4251-9A4D-BCCD-1B7EBB0436C0.root; cannot allocate memory
)

    'root://cmsxrootd.fnal.gov//store/data/Run2016D/JetHT/NANOAOD/HIPM_UL2016_MiniAODv2_NanoAODv9-v2/2530000/B091446A-4251-9A4D-BCCD-1B7EBB0436C0.root'

Files may be specified as:
   * str/bytes: relative or absolute filesystem path or URL, without any colons
         other than Windows drive letter or URL schema.
         Examples: "rel/file.root", "C:\abs\file.root", "http://where/what.root"
   * str/bytes: same with an object-within-ROOT path, separated by a colon.
         Example: "rel/file.root:tdirectory/ttree"
   * pathlib.Path: always interpreted as a filesystem path or URL only (no
         object-within-ROOT path), regardless of whether there are any colons.
         E

Output()

Filename:  root://cmsxrootd.fnal.gov//store/data/Run2018B/JetHT/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v1/130000/409CD910-965A-834D-A823-B0CBF46CCB68.root
Lenght of events  95906 length of weights  95906
IOV  2018 , era  Run2018B
apply lumimask
call applyprescales
Trigger year  2018
Trigger: len of events  63707 len of weights  63707
[3.4e+04, 128, 128, 3.4e+04, 3.4e+04, 350, ... 350, 48, 3.4e+04, 3.4e+04, 3.4e+04]
Npvs  ['ndof', 'x', 'y', 'z', 'chi2', 'score', 'npvs', 'npvsGood']
Reco sel: len of events  1147 len of weights  1147
Reco kine/topo sel: len of events  1094 len of weights  1094
FatJet:  [[750, 733], [754, 701], [382, 340], [389, ... [336, 226], [758, 752], [369, 309]]
Fatjet flattened along axis 0:  [750, 733, 754, 701, 382, 340, 389, 363, ... 670, 648, 336, 226, 758, 752, 369, 309]
Length of FatJet after falttenting:  1094
Filename:  root://cmsxrootd.fnal.gov//store/data/Run2018B/JetHT/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v1/130000/409CD910-965A-834D-A823-B0CBF46CCB68.root
Leng

...

Filename:  root://cmsxrootd.fnal.gov//store/data/Run2017C/JetHT/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/70000/4135193D-CE12-1245-AE35-B7DA90E76040.root
Lenght of events  98229 length of weights  98229
IOV  2017 , era  Run2017C
apply lumimask
call applyprescales
Trigger year  2017
Trigger: len of events  46853 len of weights  46853
[8e+03, 94, 8e+03, 94, 390, 8e+03, 390, ... 54, 141, 141, 600, 3.57e+04, 141]
Npvs  ['ndof', 'x', 'y', 'z', 'chi2', 'score', 'npvs', 'npvsGood']
Reco sel: len of events  1588 len of weights  1588
Reco kine/topo sel: len of events  1494 len of weights  1494
FatJet:  [[674, 565], [884, 760], [906, 572], [751, ... [976, 554], [980, 969], [776, 683]]
Fatjet flattened along axis 0:  [674, 565, 884, 760, 906, 572, 751, 584, ... 508, 396, 976, 554, 980, 969, 776, 683]
Length of FatJet after falttenting:  1494
Filename:  root://cmsxrootd.fnal.gov//store/data/Run2017C/JetHT/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/70000/4135193D-CE12-1245-AE35-B7DA90E76040.root
Lenght of e

/opt/conda/lib/python3.8/site-packages/coffea/processor/executor.py:1360: UserWarning: XRootD error: [ERROR] Operation expired
in file root://cmsxrootd.fnal.gov//store/data/Run2017C/JetHT/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/280000/CF189D37-A355-3B4A-9E55-9664ACB42296.root
  warnings.warn(str(e))


Filename:  root://cmsxrootd.fnal.gov//store/data/Run2017C/JetHT/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/280000/CF189D37-A355-3B4A-9E55-9664ACB42296.root
Lenght of events  97161 length of weights  97161
IOV  2017 , era  Run2017C
apply lumimask
call applyprescales
Trigger year  2017
Trigger: len of events  70911 len of weights  70911
[372, 1.59e+03, 13, 2.8e+04, 2.8e+04, 39, ... 3.2e+04, 121, 121, 1, 3.2e+04, 121]
Npvs  ['ndof', 'x', 'y', 'z', 'chi2', 'score', 'npvs', 'npvsGood']
Reco sel: len of events  2874 len of weights  2874
Reco kine/topo sel: len of events  2740 len of weights  2740
FatJet:  [[860, 751], [950, 920], [750, 718], [544, ... [597, 586], [852, 729], [789, 726]]
Fatjet flattened along axis 0:  [860, 751, 950, 920, 750, 718, 544, 526, ... 1.12e+03, 597, 586, 852, 729, 789, 726]
Length of FatJet after falttenting:  2740
Filename:  root://cmsxrootd.fnal.gov//store/data/Run2017C/JetHT/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/280000/CF189D37-A355-3B4A-9E55-9664ACB42296.root
Lengh

/opt/conda/lib/python3.8/site-packages/coffea/processor/executor.py:1360: UserWarning: XRootD error: [ERROR] Operation expired
in file root://cmsxrootd.fnal.gov//store/data/Run2017C/JetHT/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/280000/7553F061-C9EF-3E44-8200-AD35D5FCA11F.root
  warnings.warn(str(e))


Filename:  root://cmsxrootd.fnal.gov//store/data/Run2017C/JetHT/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/280000/7553F061-C9EF-3E44-8200-AD35D5FCA11F.root
Lenght of events  101094 length of weights  101094
IOV  2017 , era  Run2017C
apply lumimask
call applyprescales
Trigger year  2017
Trigger: len of events  71908 len of weights  71908
[3.2e+04, 121, 3.2e+04, 46, 3.2e+04, 1.92e+03, ... 2.4e+03, 1, 3.57e+04, 54, 2.4e+03]
Npvs  ['ndof', 'x', 'y', 'z', 'chi2', 'score', 'npvs', 'npvsGood']
Reco sel: len of events  2924 len of weights  2924
Reco kine/topo sel: len of events  2777 len of weights  2777
FatJet:  [[1.32e+03, 1.18e+03], [449, 432], [449, 418], ... 752, 570], [918, 635], [845, 766]]
Fatjet flattened along axis 0:  [1.32e+03, 1.18e+03, 449, 432, 449, 418, 590, ... 728, 752, 570, 918, 635, 845, 766]
Length of FatJet after falttenting:  2777
Filename:  root://cmsxrootd.fnal.gov//store/data/Run2017C/JetHT/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/280000/7553F061-C9EF-3E44-8200-AD35D5FCA11F.r

In [ ]:
fname = 'coffeaOutput/dijetHistsTest_QCDsim_30.0_2.5.pkl'
with open(fname, "rb") as f:
    result = pickle.load( f )
# result=result[0]
import matplotlib.pyplot as plt
%matplotlib inline
#import mplhep as hep
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('Ungroomed MC jets')
result['response_matrix_u'][{'dataset':sum}].project('ptgen').plot1d(ax=axs[0,0], binwnorm=True)
result['response_matrix_u'][{'dataset':sum}].project('ptreco').plot1d(ax=axs[0,1])
result['response_matrix_u'][{'dataset':sum}].project('mgen').plot1d(ax=axs[1,0], binwnorm=True)
result['response_matrix_u'][{'dataset':sum}].project('mreco').plot1d(ax=axs[1,1])

In [ ]:
fname = 'coffeaOutput/dijetHistsTest_JetHT_30.0_2.5.pkl'
with open(fname, "rb") as f:
    result = pickle.load( f )
# result=result[0]
import matplotlib.pyplot as plt
%matplotlib inline
#import mplhep as hep
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('Ungroomed Data jets')
result['jet_pt_mass_reco_u'][{'dataset':sum}].project('ptreco').plot1d(ax=axs[0,0], binwnorm=True)
result['jet_pt_mass_reco_u'][{'dataset':sum}].project('mreco').plot1d(ax=axs[0,1])
result['jet_pt_mass_reco_g'][{'dataset':sum}].project('ptreco').plot1d(ax=axs[1,0], binwnorm=True)
result['jet_pt_mass_reco_g'][{'dataset':sum}].project('mreco').plot1d(ax=axs[1,1])
axs[0,0].set_xlim(0,1200)
axs[1,0].set_xlim(0,1200)

In [ ]:
numerator = result['jet_pt'][{'partonFlav':'Gluon'}].project('ptreco')
denominator = result['jet_pt'].project('ptreco')
print(numerator.values(), denominator.values())

numerator.label = r'$\epsilon$'
fig = plt.gcf()
grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])

main_ax = fig.add_subplot(grid[0])
subplot_ax = fig.add_subplot(grid[1], sharex=main_ax)
plt.setp(main_ax.get_xticklabels(), visible=False)
main_ax, sublot_ax = numerator.plot_ratio(
    denominator,
    rp_num_label="gluons",
    rp_denom_label="all",
    rp_uncert_draw_type="line",
    rp_uncertainty_type="efficiency",
    ax_dict = {"main_ax":main_ax,"ratio_ax":subplot_ax}
)

plt.xlabel("Pt (GeV)")
plt.ylabel("Gluon fraction")
fig.suptitle('Gluon content as a function of pT')

In [ ]:
numerator = result['jet_eta'][{'partonFlav':'Gluon'}].project('eta')
denominator = result['jet_eta'].project('eta')
print(numerator.values(), denominator.values())
numerator.label = r'$\epsilon$'
numerator.label = r'$\epsilon$'
fig = plt.gcf()
grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])

main_axis = fig.add_subplot(grid[0])
ratio_ax = fig.add_subplot(grid[1], sharex=main_axis)
plt.setp(main_axis.get_xticklabels(), visible=False)
main_ax, sublot_ax = numerator.plot_ratio(
    denominator,
    rp_num_label="gluons",
    rp_denom_label="all",
    rp_uncert_draw_type="line",
    rp_uncertainty_type="efficiency",
    ax_dict = {"main_ax":main_axis,"ratio_ax":ratio_ax}
)

plt.xlabel(r'$\eta$')
plt.ylabel("Gluon fraction")
main_axis.set_title('Gluon content as a function of eta')


In [ ]:
#sum entries in bins --> plot as y-axis
plt.rcParams["figure.figsize"] = (20,5)


fig, axs = plt.subplots(1, 3)
result['jet_mass'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[0], overlay='partonFlav')
plt.rc('legend',fontsize='small')
plt.xlabel("Jet mass (GeV)")
result['jet_pt'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[1], overlay='partonFlav')
plt.rc('legend',fontsize='small')
#axs[0].set_xlabel("Jet mass (GeV)")
plt.xlabel("Jet pT (GeV)")
result['jet_eta'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[2], overlay='partonFlav')
plt.rc('legend',fontsize='small')
plt.legend()
plt.xlabel("Jet eta (GeV)")

fig.suptitle('Dijet eta <2.5, pt> 200.', fontsize="large")

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, axs = plt.subplots(2, 3)
print(result['jet_mass'])
result['jet_mass'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,0], overlay='partonFlav')
plt.rc('legend',fontsize='small')
plt.xlabel("Jet mass (GeV)")
result['jet_pt'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,1], overlay='partonFlav')
plt.rc('legend',fontsize='small')
#axs[0].set_xlabel("Jet mass (GeV)")
plt.xlabel("Jet pT (GeV)")
result['jet_eta'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,2], overlay='partonFlav')
plt.rc('legend',fontsize='small')
plt.legend()
plt.xlabel("Jet eta (GeV)")
result['jet_mass'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,0], overlay='partonFlav')
plt.rc('legend',fontsize='small')
plt.xlabel("Jet mass (GeV)")
result['jet_pt'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,1], overlay='partonFlav')
plt.rc('legend',fontsize='small')
#axs[0].set_xlabel("Jet mass (GeV)")
plt.xlabel("Jet pT (GeV)")
result['jet_eta'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,2], overlay='partonFlav')
plt.rc('legend',fontsize='small')
plt.legend()
plt.xlabel("Jet eta (GeV)")
fig.suptitle('Dijet eta <2.5, pt> 200.', fontsize="large")

In [ ]:
njets=result['cutflow']['nJets']
ngluonjets=result['cutflow']['nGluonJets']
print(result['cutflow'])
print("Gluon purity:", ngluonjets/njets)

In [ ]:
response_matrix_u_values = result['response_matrix_u'].project("ptreco", "mreco", "ptgen", "mgen").values()
response_matrix_g_values = result['response_matrix_g'].project("ptreco", "mreco", "ptgen", "mgen").values()

In [ ]:
nptreco,nmassreco,nptgen,nmassgen = response_matrix_u_values.shape

In [ ]:
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) )
print(ak.max(response_matrix_u_final))

In [ ]:
response_matrix_g_final = response_matrix_g_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) )

In [ ]:
plt.figure(figsize = (40,80))
plt.imshow( np.log(response_matrix_u_final+1), aspect="equal", cmap="Blues" )
plt.xlabel("RECO", fontsize=50)
plt.ylabel("GEN", fontsize=50)
plt.tick_params(labelsize=40)

In [ ]:
plt.figure(figsize = (40,80))
plt.imshow( np.log(response_matrix_g_final+1), aspect="equal", cmap="Blues" )
plt.xlabel("RECO", fontsize=50)
plt.ylabel("GEN", fontsize=50)
plt.tick_params(labelsize=40)

In [ ]:
plt.figure(figsize = (40,80))
plt.imshow( response_matrix_u_final, aspect="equal", cmap="Blues" )
plt.xlabel("RECO", fontsize=50)
plt.ylabel("GEN", fontsize=50)
plt.tick_params(labelsize=40)

## 